In [ ]:
pip install PyMySQL

In [ ]:
conda config --add channels conda-forge

In [17]:
conda update conda=3.7

Traceback (most recent call last):


Note: you may need to restart the kernel to use updated packages.



  File "D:\anaconda\anaconda\Scripts\conda-script.py", line 11, in <module>
    from conda.cli import main
ModuleNotFoundError: No module named 'conda'


In [3]:
conda update  --all


Note: you may need to restart the kernel to use updated packages.


Traceback (most recent call last):
  File "D:\anaconda\anaconda\Scripts\conda-script.py", line 11, in <module>
    from conda.cli import main
ModuleNotFoundError: No module named 'conda'


In [4]:
SITE_PACKAGES=/home/himal/miniconda3/lib/python3.7/site-packages

SyntaxError: invalid syntax (<ipython-input-4-e7fe7857abca>, line 1)

In [2]:
conda install -c conda-forge/label/gcc7/win-64 scikit-surprise


Note: you may need to restart the kernel to use updated packages.


Traceback (most recent call last):
  File "D:\anaconda\anaconda\Scripts\conda-script.py", line 11, in <module>
    from conda.cli import main
ModuleNotFoundError: No module named 'conda'


In [ ]:
conda install -c conda-forge scikit-surpirse

In [ ]:
import pandas as pd
import numpy as np

import pymysql
import pymysql.cursors

In [ ]:
keyword_except_condition = np.array(['까페', '감성주점', '키즈카페', '정종/대포집/소주방', 
                           '전통찻집', '출장조리', '라이브카페'])

keyword_except_name = np.array(['공차', '이디야', '투썸', '탐앤탐스', '이삭토스트', 
                           '한솥', '까페', '카페', '커피', 'COFFEE', '랑데자뷰', '빽다방', 
                           '감성커피', '아마스빈' ,'쥬씨' ,'차얌'])


In [ ]:
restaurant = "-gu_pre.csv"
address_list = ['Seodaemun']



# Connect to the database
connection = pymysql.connect(host='db-sang-diary.cdjhmqsvlizr.us-east-2.rds.amazonaws.com',
                             user='root',
                             password='akslzj19!',
                             db='myLunch',
                             charset='utf8mb4',
                             cursorclass=pymysql.cursors.DictCursor)


for i in address_list:
    fileName = i + restaurant
    
    temp = pd.read_csv(fileName, encoding="cp949")
    
    data_res = {'상세영업상태명': temp['상세영업상태명'],
            '소재지면적': temp['소재지면적'],
            '지번주소' : temp['지번주소'],
            '사업장명' : temp['사업장명'],
            '업태구분명' : temp['업태구분명']
           }

    data_res = pd.DataFrame(data_res)

    data_res = PretreatmentData(data_res)
    
    insertResData(i, data_res)
# 상세영업상태명 / 전화번호 / 소재지면적 / 지번주소 / 사업장명 / 업태구분명

In [ ]:
def PretreatmentData(data_res):
    
    for i in range (len(data_res)-1, -1, -1):
        #폐업 제외
        if data_res['상세영업상태명'][i] == "폐업":
            data_res = data_res.drop(data_res.index[i])
        # 업태 구분명 제외
        data_res = data_res.reset_index(drop=True)
        for j in range (0, len(keyword_except_condition)):
            data_res = data_res.reset_index(drop=True)
            if data_res['업태구분명'][i] == keyword_except_condition[j]:
            #if data_res['업태구분명'][i] == '까페':
                data_res = data_res.drop(data_res.index[i])
        # 사업장명 기준 제외
        data_res = data_res.reset_index(drop=True)
        for j in range (0, len(keyword_except_name)):
            data_res = data_res.reset_index(drop=True)
            if data_res['사업장명'][i].find(keyword_except_name[j]) != -1:
            #if data_res['업태구분명'][i] == '까페':
                data_res = data_res.drop(data_res.index[i])

    data_res.drop(['상세영업상태명'], axis = 1, inplace = True)
    
    return data_res

In [ ]:
temp = pd.read_csv(restaurant, encoding="cp949")

data_res = {'상세영업상태명': temp['상세영업상태명'],
            '전화번호': temp['전화번호'],
            '소재지면적': temp['소재지면적'],
            '지번주소' : temp['지번주소'],
            '사업장명' : temp['사업장명'],
            '업태구분명' : temp['업태구분명']
           }

data_res = pd.DataFrame(data_res)

for i in range (len(data_res)-1, -1, -1):
    if data_res['상세영업상태명'][i] == "폐업":
        data_res = data_res.drop(data_res.index[i])


for i in range (len(data_res)-1, -1, -1):
    data_res = data_res.reset_index(drop=True)
    for j in range (0, len(keyword_except)):
        data_res = data_res.reset_index(drop=True)
        if data_res['업태구분명'][i] == keyword_except[j]:
        #if data_res['업태구분명'][i] == '까페':
            data_res = data_res.drop(data_res.index[i])
            
for i in range (len(data_res)-1, -1, -1):
    data_res = data_res.reset_index(drop=True)
    for j in range (0, len(keyword_except)):
        data_res = data_res.reset_index(drop=True)
        if data_res['사업장명'][i].find(keyword_except[j]) != -1:
        #if data_res['업태구분명'][i] == '까페':
            data_res = data_res.drop(data_res.index[i])
            
data_res.drop(['상세영업상태명'], axis = 1, inplace = True)

In [ ]:
def insertResData(address, data_res):

    data_res = data_res.dropna(axis=0)
    data_res = data_res.reset_index(drop=True)

    tableName = "restaurant_" + address

    
    try:
        with connection.cursor() as cursor:

            sql =   "Create Table "+tableName +" (id int AUTO_INCREMENT, address varchar(30), restaurantName varchar(20), restaurantCondition varchar(10), primary key(id)); "

            cursor.execute(sql)
            connection.commit()

            
            for i in range(0, len(data_res)):
                # Create a new record
                sql = "INSERT INTO "+" "+ tableName +" "+ " (address, restaurantName, restaurantCondition) VALUES (%s, %s, %s);"
                cursor.execute(sql, (data_res['지번주소'][i], data_res['사업장명'][i], data_res['업태구분명'][i]))
                connection.commit()
            
    finally:
        connection.close()